In [ ]:
'''Внимание, в программе обращения к mystem составлены для операционной системы Linux.
Если программа не размечает файлы — исправьте соответствующие запросы или запустите программу на Linux.

В папке с программой должен содержаться mystem.
Кроме того, для корректной работы программы долна быть установлена сторонняя библиотека bs4.'''

In [ ]:
import urllib.request
import bs4
import os
import shutil

In [ ]:
def get_pages():
    for a in range(10000, 22930, 10):
        url = 'https://moe-online.ru/news/society/10%d' % a
        req = urllib.request.Request(url)
        try:
            with urllib.request.urlopen(req) as response:
               page = response.read().decode('utf-8')
        except urllib.error.HTTPError:
            continue
        if 'Ошибка 404' in page:
            continue
        yield page, url

def read_page(page):
    tree = bs4.BeautifulSoup(page,'html.parser')
    scripts = tree.findAll('script')
    [script.extract() for script in scripts]
    st_name = tree.find_all('h1', {'class': 'st-name'})[0].get_text()
    date = tree.find_all('span', {'class': 'time'})[0].get_text()[:10]
    cat = tree.find_all('a', {'class': 'rubrika more_screen'})[0].get_text()
    text = tree.find_all('div', {'class': 'font_os app_in_text'})[0].get_text().replace(u'\xa0', u' ')
    return st_name.strip(), date, cat, text

def create_path(date, counter, path_type):
    path = os.path.join('газеты', path_type, date[6:10], date[3:5])
    if not os.path.exists(path):
        os.makedirs(path)
        counter = 0
    path = os.path.join(path, str(counter) + '.txt')
    return path, counter

def write_plain(counter, url, st_name, date, cat, text):
    path, counter = create_path(date, counter, 'plain')    
    with open(path, 'w', encoding='utf-8') as f:
        f.write('@au Noname\n@ti %s\n@da %s\n@topic %s\n@url %s\n%s' % (st_name, date, cat, url, text))
    return counter
        
def write_mystem_xml(counter, date):
    path, counter = create_path(date, counter, 'mystem-xml') 
    os.system('./mystem -id --format xml tmp.txt ' + path)
    return counter

def write_mystem_plain(counter, date):
    path, counter = create_path(date, counter, 'mystem-plain') 
    os.system('./mystem -id tmp.txt ' + path)
    return counter
    
def csv_string(counter, url, st_name, date, cat, text):
    path, counter = create_path(date, counter, 'plain')    
    feature_list = [path, '', '', '', st_name, date, 'публицистика', '',
                    '', cat, '', 'нейтральный', 'н-возраст', 'н-уровень', 'городская', url, 'МОЁ! Online', '', 
                    date[6:10], 'газета', 'Россия', 'Воронежская область', 'ru']
    return '\t'.join(feature_list), counter
    
def main():
    if os.path.exists('газеты'):
        shutil.rmtree('газеты')
    counter = 0
    os.makedirs('газеты')
    with open(os.path.join('газеты', 'metadata.csv'), 'w', encoding='utf-8') as csv:
        for page, url in get_pages():
            st_name, date, cat, text = read_page(page)

            string, counter = csv_string(counter, url, st_name, date, cat, text)
            csv.write(string +'\n')

            with open('tmp.txt', 'w', encoding='utf-8') as tmp:
                tmp.write(text)

            counter = write_plain(counter, url, st_name, date, cat, text)
            counter = write_mystem_plain(counter, date)
            counter = write_mystem_xml(counter, date)

            counter += 1
    os.remove('tmp.txt')

In [ ]:
main()